# Single file downloads

In [1]:
import requests
import os
import time
import random

def get_emsr_numbers(query):
    """
    Retrieves EMSR numbers for the given query (Country / City / Disaster Type).\n
    Returns a list of all disaster codes for a specific query

    """
    query = query.lower()
    disaster_list = []

    url = f"https://mapping.emergency.copernicus.eu/activations/api/activations/?categories=fire%2Cvolcan%2Cearthquake%2Cmass%2Cstorm%2Cflood%2Chumanitarian%2Cindustrial%2Cenvironment%2Cother&activationTime=2012-01%2C2025-2&countries=&services=&closed=&limit=500&q={query}"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        if "results" in data:
            results = data["results"]
            if results:
                for item in results:
                    if "code" in item and item["code"].startswith("EMSR"):
                        emsr_number = item["code"].replace("EMSR", "")
                        disaster_list.append(emsr_number)
            return disaster_list
        else:
            return []
    except requests.exceptions.RequestException as e:  # Catch specific request exceptions
        print(f"Error retrieving EMSR numbers: {e}")
        return None

def crawl_emsr_data(code, download_path, retry_attempts=3, initial_retry_delay=2, max_retry_delay=15):
    """Crawls and downloads data for a given EMSR number."""
    url = f"https://rapidmapping.emergency.copernicus.eu/backend/dashboard-api/public-activations/?code=EMSR{code}"

    for attempt in range(retry_attempts):
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()

            if data["results"]:
                activation_name = data["results"][0]["name"].replace(" ", "_").replace(",", "")
                for city in data["results"][0]["aois"]:
                    for product in city["products"]:
                        if product["type"] == "GRA" and product["downloadPath"]:
                            download_url = product["downloadPath"]
                            filename = download_url.split("/")[-1]
                            new_filename = f"{activation_name}_{filename}"
                            filepath = os.path.join(download_path, new_filename)

                            try:
                                response = requests.get(download_url, stream=True)
                                response.raise_for_status()

                                with open(filepath, 'wb') as f:
                                    for chunk in response.iter_content(chunk_size=16384):
                                        if chunk:
                                            f.write(chunk)

                                print(f"Downloaded: {new_filename}")

                            except requests.exceptions.HTTPError as e:
                                if e.response.status_code == 403:
                                    print(f"403 Forbidden error for {new_filename}.")
                                elif e.response.status_code == 429:
                                    print(f"Throttle error when downloading {new_filename}.")
                                else:
                                    print(f"HTTP Error for {new_filename}: {e}")
                            except requests.exceptions.RequestException as e:
                                print(f"Error downloading '{new_filename}': {e}")
                return True
            else:
                print(f"Nothing found within Results for EMSR{code}")
                return True

        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 429:
                delay = min(initial_retry_delay * (1.5 ** attempt), max_retry_delay) + random.uniform(0.5, 1.5)
                print(f"Throttle error for EMSR{code}. Retrying in {delay:.2f} seconds...")
                time.sleep(delay)

            elif e.response.status_code == 403:
                print(f"403 Forbidden error for EMSR{code}. Skipping to next file.")
                return False
            else:
                print(f"HTTP Error for EMSR{code}: {e}")
                return False

        except requests.exceptions.RequestException as e:
            delay = min(initial_retry_delay * (1.5 ** attempt), max_retry_delay) + random.uniform(0.5, 1.5)
            print(f"Error making the API request for EMSR{code}. Retrying in {delay:.2f} seconds...")
            time.sleep(delay)

    print(f"Failed to process EMSR{code} after {retry_attempts} tries.")
    return False

def crawl_for_data(user_defined_country, dwnld_path):
    """Crawls for data using the provided country and tracks statistics."""
    emsr_numbers = get_emsr_numbers(user_defined_country)

    if emsr_numbers is None:
        print("Could not return any EMSR codes, please ensure the selected country has valid EMSR codes")
        return

    print(f"Total search results: {len(emsr_numbers)}")

    for c in emsr_numbers:
        crawl_emsr_data(code=c, download_path=dwnld_path)


In [5]:
download_dir = os.path.expanduser("~/Downloads") 
crawl_for_data(input("Enter the search criteria (Country / City / Disaster Type): "), dwnld_path = download_dir)

Total search results: 76
Downloaded: Flood_in_Valencia_Region_Spain_EMSR773_AOI03_GRA_PRODUCT_v2.zip
Downloaded: Flood_in_Valencia_Region_Spain_EMSR773_AOI05_GRA_PRODUCT_v1.zip
Downloaded: Flood_in_Valencia_Region_Spain_EMSR773_AOI07_GRA_PRODUCT_v1.zip
Downloaded: Flood_in_Valencia_Region_Spain_EMSR773_AOI08_GRA_PRODUCT_v1.zip
Downloaded: Flood_in_Valencia_Region_Spain_EMSR773_AOI09_GRA_PRODUCT_v1.zip
Downloaded: Flood_in_Valencia_Region_Spain_EMSR773_AOI10_GRA_PRODUCT_v1.zip
Downloaded: Flood_in_Valencia_Region_Spain_EMSR773_AOI12_GRA_PRODUCT_v1.zip
Downloaded: Flood_in_Valencia_Region_Spain_EMSR773_AOI13_GRA_PRODUCT_v1.zip
Downloaded: Flood_in_Valencia_Region_Spain_EMSR773_AOI14_GRA_PRODUCT_v1.zip
Downloaded: Flood_in_Valencia_Region_Spain_EMSR773_AOI14_GRA_MONIT01_v1.zip
Downloaded: Flood_in_Valencia_Region_Spain_EMSR773_AOI15_GRA_PRODUCT_v1.zip
Downloaded: Flood_in_Valencia_Region_Spain_EMSR773_AOI16_GRA_PRODUCT_v1.zip
Downloaded: Flood_in_Valencia_Region_Spain_EMSR773_AOI17_GRA_PR